In [ ]:
import os
import torch
from utils import videos_to_frames, extract_faces, interpolate_missing_frames, resize_excess, pad_resize, save_as_tensor
from torch.utils.data import DataLoader

In [ ]:
from model import FakeNet
model = FakeNet()
model = torch.nn.DataParallel(model)
print(model)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
load_name = os.path.join('fake_class2_29_checkpoint.pth.tar')
loaded_model = torch.load(load_name, map_location=torch.device('cpu'))
model.load_state_dict(loaded_model['state_dict'])
#optimizer.load_state_dict(loaded_model['optimizer'])
start_epoch = loaded_model['epoch']

### Run following lines to test on full data
If you don't want please skip, datasize is approximately 4GB

In [ ]:
%env KAGGLE_USERNAME=""
%env KAGGLE_KEY=""

In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

In [ ]:
!unzip deepfake-detection-challenge.zip

In [ ]:
with open(os.path.join("deepfake-detection-challenge", "train_sample_videos", "metadata.json"), 'r') as j:
    metadata = j.read()

filedata = metadata.replace('"REAL"', '1')
filedata = filedata.replace('"FAKE"', '0')

with open(os.path.join("deepfake-detection-challenge", "train_sample_videos", "metadata_edited.json"), 'w') as file:
  file.write(filedata)

### Following lines preprocess the data
- demo_data_path should be the name of folder which encloses videos
- If you want to preprocess videos and just want to use .pt tensors skip this section.
- At the end of this section you should have .pt tensors saved inside the demo_data_path variable

In [ ]:
demo_data_path = "demo_data"

In [ ]:

videos_to_frames(demo_data_path)
extract_faces(demo_data_path)
interpolate_missing_frames(demo_data_path)
resize_excess(demo_data_path)
pad_resize(demo_data_path)

In [ ]:
save_as_tensor(demo_data_path)

### Evaluation

In [ ]:
# This variable should be the path which encloses the .pt tensor
demo_data_path = "demo_data"

In [ ]:
from data_loader import FakeDetectDataset
dataset = FakeDetectDataset(demo_data_path)
data_loader_params = {'dataset': dataset, 'batch_size': 1, 'shuffle': False, 'sampler': None,
                        'batch_sampler': None, 'num_workers': 0, 'collate_fn': None}
eval_loader = DataLoader(**data_loader_params)

In [ ]:
model.eval()
with torch.no_grad():
    acc = 0
    for (batch_no, sample) in enumerate(eval_loader):
        inp = sample[0]
        label = torch.tensor(sample[1], dtype=torch.float)
        output = model(inp)
        output = output.squeeze()
        number_to_probs = lambda class_int: "REAL prob: {:.2f} FAKE probability: {:.2f}".format(torch.sigmoid(output), 1-torch.sigmoid(output).item()) 
        number_to_class = lambda class_int: "REAL" if  class_int > 0.5 else "FAKE"
        print(output)
        class_pred_prob = torch.sigmoid(output)
        class_pred = 1 if class_pred_prob.item() > 0.5 else 0
        if class_pred == label.item():
            acc += 1
        print(f"Prediction = \n\t{number_to_probs(class_pred_prob.item())}")
        print(f"Actual = \n\t{number_to_class(label.item())}")
    print("Accuracy is " + str( acc / (batch_no +1)))